In [10]:
import pandas as pd

df = pd.read_csv("Reddit_macron.csv", encoding='utf-8') # ouvrir le fichier csv

#Créer un copie de Travail en choisissant uniquement les colonnes qui nous intéressent
df_nd = df[['id','author','title','Text','subreddit','created','score']].copy()
df_nd = df_nd.drop_duplicates().copy()
df_nd.head()

,id,author,title,Text,subreddit,created,score
0,1jk8ild,Nohan07,Macron propose d'expulser les locataires des H...,NaN,FranceInsoumise,2025-03-26 11:22:51,2
1,1j6cwa4,Nohan07,Ukraine - Russie : Macron fait n'importe quoi !,NaN,FranceInsoumise,2025-03-08 09:34:16,2
2,1iuxpxt,Nohan07,Le pacte secret entre Macron et Le Pen ! – Pan...,NaN,FranceInsoumise,2025-02-21 19:20:55,3
3,1it9v25,Nohan07,Marine Le Pen a soutenu un ami de Macron ￼,NaN,FranceInsoumise,2025-02-19 17:39:24,5
4,1imtylh,Nohan07,"Fichage, surveillance, traçage : le gouvernent...",NaN,FranceInsoumise,2025-02-11 09:22:52,5


In [11]:
import datetime
import time
import os
import calendar

df_nd.loc[:, 'year'] = pd.to_datetime(df_nd.created, format='%Y-%m-%d %H:%M:%S').dt.year
df_nd.loc[:, 'month'] = pd.to_datetime(df_nd.created, format='%Y-%m-%d %H:%M:%S').dt.month
df_nd['month_name'] = df_nd["month"].apply(lambda x: calendar.month_abbr[x])
cols=["month_name","year"]
df_nd['period'] = df_nd[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
df_nd.head()

,id,author,title,Text,subreddit,created,score,year,month,month_name,period
0,1jk8ild,Nohan07,Macron propose d'expulser les locataires des H...,NaN,FranceInsoumise,2025-03-26 11:22:51,2,2025,3,Mar,Mar-2025
1,1j6cwa4,Nohan07,Ukraine - Russie : Macron fait n'importe quoi !,NaN,FranceInsoumise,2025-03-08 09:34:16,2,2025,3,Mar,Mar-2025
2,1iuxpxt,Nohan07,Le pacte secret entre Macron et Le Pen ! – Pan...,NaN,FranceInsoumise,2025-02-21 19:20:55,3,2025,2,Feb,Feb-2025
3,1it9v25,Nohan07,Marine Le Pen a soutenu un ami de Macron ￼,NaN,FranceInsoumise,2025-02-19 17:39:24,5,2025,2,Feb,Feb-2025
4,1imtylh,Nohan07,"Fichage, surveillance, traçage : le gouvernent...",NaN,FranceInsoumise,2025-02-11 09:22:52,5,2025,2,Feb,Feb-2025


In [12]:
mandat_year=[]
for i in df_nd.index:
    if df_nd['month'][i] >= 5:
        mandat_year.append(df_nd['year'][i]-2017)
    else:
        mandat_year.append(df_nd['year'][i]-2018)

In [13]:
df_nd['mandat'] = mandat_year
df_nd.groupby('mandat').nunique()

,id,author,title,Text,subreddit,created,score,year,month,month_name,period
mandat,,,,,,,,,,,
4,119,15,119,11,3,119,24,2,8,8,8
5,251,24,249,1,5,250,26,2,12,12,12
6,433,32,422,2,6,433,31,2,12,12,12
7,201,34,197,0,6,201,61,2,12,12,12


In [14]:
df_nd.tail()

,id,author,title,Text,subreddit,created,score,year,month,month_name,period,mandat
999,1847pf8,konbini,"Gérard Collomb, ancien maire de Lyon et ancien...",NaN,franceinfo,2023-11-26 11:16:42,0,2023,11,Nov,Nov-2023,6
1000,154qm6b,konbini,"Alors, ça dit quoi ce remaniement d’Emmanuel M...",NaN,franceinfo,2023-07-20 15:14:05,0,2023,7,Jul,Jul-2023,6
1001,123sc94,DrogDrill,Il faut faire tomber le gouvernement Macron!,NaN,franceinfo,2023-03-27 18:31:34,1,2023,3,Mar,Mar-2023,5
1002,yn0om1,infofrance,Emmanuel Macron ne sera pas au départ de la Ro...,NaN,franceinfo,2022-11-05 18:36:54,1,2022,11,Nov,Nov-2022,5
1003,y72b5s,konbini,"Macron félicite Benzema sur Twitter, la France...",NaN,franceinfo,2022-10-18 11:38:11,0,2022,10,Oct,Oct-2022,5


In [15]:
list_period=df_nd["period"].drop_duplicates().tolist()
list_period

list_posts =df_nd['title'].tolist()
# créer une liste regroupant tous les posts par mois 
Collection = [ ]
for p in list_period:
    Collection.append(df_nd['title'][df_nd["period"] == p].tolist())
    
len(Collection)
print(Collection[0][0])

Macron propose d'expulser les locataires des HLM et tente de les faire passer pour des privilégiés.


# Pre-Processing Corpus
Les prétraitement de votre corpus va dépendre en grande partie à la qualité de vos données.
Il faut toutefois garder à l’esprit que certains des prétraitement ne sont pas nécessaires selon le modèle de machine learning que vous utiliserez par la suite ! 

In [16]:
# Load packages
import nltk
#nltk.download('punkt')
#nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words_english  = set(stopwords.words('english'))
stop_words_french   = set(stopwords.words('french'))
stop_words_specific = ["'", "le","ce","de","cest","celui","ne"]
stop_words          = set(stopwords.words('french') + stopwords.words('english') + stop_words_specific)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## étape 1 : Basic Cleaning - à adapter à votre corpus/imagination !

- Remove Unicode Strings and Noise
- Remove/Replace URLs, User Mentions and Hashtags
- Non-Letter characters: numbers, emojis, or hash marks.
- Remove/Replace Slang and Abbreviations
- Remove/Replace Contractions
- Remove/Replace Numbers
- Remove/Replace Repetitions of Punctuation
- Remove Punctuation
- Handling Capitalized Words / Lowercase
- Replace Elongated Words (ex: hahahaaaa, ‘Duuuuude, that's awful,’”)

https://pynative.com/python-regex-replace-re-sub/

In [17]:
#the following pre-tokenization receives string as input parameter
#and returns string as output
import re
#import contractions

def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet) # remove Twitter links
    return tweet

def remove_tags(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    tweet = re.sub('RT @[\w_]+:','', tweet)  # remove retweet label
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

punctuation = '!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@``'
def remove_nonText(tweet):
    tweet = re.sub('[' + punctuation + ']+', '', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub(r'\n','', tweet)  # remove escape sequence
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet) # un exemple d'image que vous pouvez compléter !

    return tweet

#def remove_contraction(text):
  #  return ' '.join([contractions.fix(word) for word in text.split()])

def pretokenization_cleaning(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_links(tweet)
    tweet = remove_tags(tweet)
    tweet = remove_users(tweet)
    tweet = remove_nonText(tweet)
#    tweet = remove_contraction(tweet) # English only !
    return tweet

In [18]:
Text=Collection[0][0:10]
print(Text)

["Macron propose d'expulser les locataires des HLM et tente de les faire passer pour des privilégiés.", "Ukraine - Russie : Macron fait n'importe quoi !", 'Allocution du Président de la République Emmanuel Macron - 05/03/2025', "Emmanuel Macron a-t-il eu une influence sur l'arrivée de soldats russes en Centrafrique ?", 'Charles de Courson : Macron cauchemarde, la France résiste ? [EN DIRECT]', '"Il veut nous faire peur" : la classe politique réagit à l\'allocution d\'Emmanuel Macron : Actualités - Orange', 'Qui est Candace Owens, l’influenceuse trumpiste qui relaie l’infox transphobe sur Brigitte Macron ?', 'Face à la “menace russe”, Macron propose d’ouvrir le débat sur un parapluie nucléaire européen', "Dissuasion nucléaire: les propos d'Emmanuel Macron braquent les autorités russes", 'Blanchiment d’argent russe : des perquisitions menées chez l’associé de Tiphaine Auzière, fille de Brigitte Macron']


#### Librarie Spacy 
SpaCy est une bibliothèque Python open source pour le traitement du texte et des langues naturelles. Créé par l’équipe de la startup Explosion AI, spaCy a été publiée sous la licence MIT en 2015 et ses principaux développeurs sont Matthew Honnibal et Ines Montani, les fondateurs de la startup.
- Contrairement à NLTK, qui est largement utilisé pour l’enseignement et la recherche, spaCy est conçu pour l’utilisation de production sur de grandes quantités de textes avec une excellente performance et précision
- Il prend en charge des modèles statistiques pour 21 langues dont le français, l’anglais, l’allemand, l’espagnol, l’italien, le portugais et le néerlandais. 
- Vous pouvez écouter le tuto de Clément Plancq ! https://github.com/clement-plancq/tuto-mate

In [19]:
# installer Spacy dans terminal
#conda install -c conda-forge spacy
#!pip install -U spacy

In [20]:
#!python3 -m spacy download fr_core_news_md
#!python -m spacy download fr_core_news_md
#conda install -c conda-forge spacy-model-fr_core_news_md

## étape 2 : Normalising data  - à adapter à votre corpus 
- Spelling Correction
- Replace Negations with Antonyms
- Handling Capitalized Words
- Lowercase
- Tokenization
- Remove Stopwords (ex: the, and….)
- Stemming / racinisation (optional)
- Lemmatizing (optional)

In [21]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import SnowballStemmer
import spacy
from spacy.lang.fr.examples import sentences 
stemmer = SnowballStemmer(language='french')
nlp = spacy.load("fr_core_news_md")

def tokenize(text):
    tknzr = TweetTokenizer(reduce_len=True)
    return tknzr.tokenize(text)

def remove_stopwords(text):
#    return " ".join([token for token in text if token.lower() not in stop_words])
    return [token for token in text if token.lower() not in stop_words_french]

def stemming(text):
    return [stemmer.stem(token) for token in text]

def lemmatizing(text):
    Text = " ".join([token for token in text])
    doc = nlp(Text)
    return [token.lemma_ for token in doc]

ModuleNotFoundError: No module named 'spacy'

In [ ]:
print(lemmatizing(remove_stopwords(tokenize(Text))))

In [ ]:
# to add stopwors to your list 
#stop_words_french

In [ ]:
def preprocess_post(post):
    """Main master function to clean and normalizing posts, and tokenizing / Lemmatization"""
    tweet = remove_links(post)
    tweet = remove_tags(post)
    tweet = remove_users(post)
    tweet = remove_nonText(post)
 #   tweet = remove_contraction(tweet) # English only
    tweet = tweet.lower()  # lower case
    tweet = tokenize(post)  # apply tokenization
    tweet = remove_stopwords(post)
#    tweet = stemming(tweet) # Optionnel 
    tweet = lemmatizing(tweet) # Optionnel
    return post

In [ ]:
# créer une liste regroupant tous les posts
list_posts_clean = [ ]
for post in list_posts:
    list_posts_clean.append(tokenize(preprocess_post(post)))

In [ ]:
list_posts_clean[0][0:100]

In [ ]:
# lets find the most frequent words
from nltk.probability import FreqDist
from collections import Counter
#Create a list of all words
all_tokens_clean = sum(list_posts_clean, [])
fdist_normalized = FreqDist(all_tokens_clean)

In [ ]:
print( "Top 50 in all Macron posts : ---" , fdist_normalized.most_common(50))  

In [ ]:
# créer une liste regroupant tous les textes par mois
Collection_clean = [ ]

for i in range(len(Collection)):
    Collection_clean.append([tokenize(preprocess_post(text)) for text in Collection[i]])

In [ ]:
for text in Collection_clean[0][0:10]:
    print(text)

In [ ]:
# lets find the most frequent words
for i in range(len(Collection_clean)):
    all_tokens_clean_temp = sum(Collection_clean[i], [])
    fdist_normalized_temp = FreqDist(all_tokens_clean_temp)
    print( "mois de", list_period[i] ," : ---" , )
    print(fdist_normalized_temp.most_common(10))               

# Approche contextuelle : les n-gramms
L’approche contextuelle : on s’intéresse non seulement aux mots et à leur fréquence, mais aussi aux mots qui suivent.
Le calcul de n-grams (bigrams pour les co-occurences de mots deux-à-deux, tri-grams pour les co-occurences trois-à-trois, etc.) constitue la méthode la plus simple pour tenir compte du contexte.

### a - Co-occurences
Une cooccurrence est une combinaison de mots qui apparaissent fréquemment ensemble dans un corpus. Ainsi on retrouve dans les même textes les termes : « professeurs, étudiants », « médecin, infirmière », ou encore « pain, boulangerie »...
Les collocations sont eux une forme privilégiée de cooccurrence : ce sont tout simplement des mots qui tendent à apparaître ensemble. Parmi les collocations en langues française, nous pouvons citer : « un désir ardent », « un léger frisson », ou encore « un regret amère ». 

In [22]:
text = nltk.Text(all_tokens_clean)
text.concordance("retraites")

NameError: name 'all_tokens_clean' is not defined

In [ ]:
# Frequent collocations in the text (usually meaningful phrases)
text.collocations()

In [ ]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

# Ngrams with the specific name "guerre"
name_filter = lambda *w: 'guerre' not in w

## Bigrams
finder = nltk.BigramCollocationFinder.from_words(text)

# only bigrams that contain 'guerre'
finder.apply_ngram_filter(name_filter)

In [ ]:
# return the 10 bi-grams with the highest student ratio
print(finder.nbest(bigram_measures.student_t,10))

In [ ]:
# return the 10 bi-grams with the highest chi2 ratio
print(finder.nbest(bigram_measures.chi_sq,10))

In [ ]:
# return the 10 bi-grams with the highest likelihood_ratio
print(finder.nbest(bigram_measures.likelihood_ratio,10))

In [ ]:
# return the 10 bi-grams with the highest PMI ratio
print(finder.nbest(bigram_measures.pmi,10))

In [ ]:
#creer une liste pour afficher les résultats en tableau
list_bigrams=[]
list_bigrams.append(finder.nbest(bigram_measures.student_t,10))
list_bigrams.append(finder.nbest(bigram_measures.chi_sq,10))
list_bigrams.append(finder.nbest(bigram_measures.likelihood_ratio,10))
list_bigrams.append(finder.nbest(bigram_measures.pmi,10))

In [ ]:
df = pd.DataFrame(list_bigrams)
df=df.transpose()
df.columns = ['Student_t', 'Chi_sq', 'Likelihood_ratio','PMI']
df

### b- Explore Co-occurring Words (Bigrams)
Une cooccurrence est une combinaison de mots qui apparaissent fréquemment ensemble dans un corpus. Ainsi on retrouve dans les même textes les termes : « professeurs, étudiants », « médecin, infirmière », ou encore « pain, boulangerie »...


In [ ]:
from nltk import bigrams
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx as nx

In [ ]:
# Create list of lists containing bigrams in tweets
terms_bigram = list(bigrams(all_tokens_clean))
# View bigrams 
terms_bigram[0:10]

In [ ]:
# Create counter of words in clean bigrams
bigram_counts = collections.Counter(terms_bigram)

bigram_counts.most_common(50)

In [ ]:
bigram_df = pd.DataFrame(bigram_counts.most_common(100),
                             columns=['bigram', 'count'])

bigram_df.head()

## c- Visualize Networks of Bigrams


In [ ]:
# Create dictionary of bigrams and their counts
d = bigram_df.set_index('bigram').T.to_dict('records')

In [ ]:
# Create network plot 
G = nx.Graph()

# Create connections between nodes
for k, v in d[0].items():
    G.add_edge(k[0], k[1], weight=(v * 10))

G.add_node("macron", weight=100)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

pos = nx.spring_layout(G, k=2)

# Plot networks
nx.draw_networkx(G, pos,
                 font_size=12,
                 width=2,
                 edge_color='grey',
                 node_color='blue',
                 node_size=50,
                 with_labels = False,
                 ax=ax)

# Create offset labels
for key, value in pos.items():
    x, y = value[0]+.135, value[1]+.145
    ax.text(x, y,
            s=key,
            bbox=dict(facecolor='lightblue', alpha=0.25),
            horizontalalignment='center', fontsize=8)
    
plt.show()

# 6 - The Similarity Between Documents

## a - Euclidean Distance
Euclidean Distance is probably one of the most known formulas for computing the distance between two points applying the Pythagorean theorem. To get it you just need to subtract the points from the vectors, raise them to squares, add them up and take the square root of them. Did it seem complex? Don’t worry, in the image below it will be easier to understand.
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

df_similarity_euclidean = pd.DataFrame(euclidean_distances(vectors, vectors))

In [ ]:
df_similarity_euclidean.head()

In [ ]:
mandat = 2
Similar_years=pd.DataFrame()
print("We look for the most similar posts to: ")
Similar_years ['Euclidean similarity'] = pd.DataFrame(df_similarity_euclidean[mandat])
Similar_years

In [ ]:
Similar_years.sort_values(by=['Euclidean similarity'] ,ascending=True)

## b - Manhattan Distance
Although Manhattan distance seems to work okay for high-dimensional data, it is a measure that is somewhat less intuitive than euclidean distance, especially when using in high-dimensional data.

When your dataset has discrete and/or binary attributes, Manhattan seems to work quite well since it takes into account the paths that realistically could be taken within values of those attributes. 

In [ ]:
from sklearn.metrics.pairwise import manhattan_distances

df_similarity_manhattan = pd.DataFrame(manhattan_distances(vectors, vectors))

In [ ]:
df_similarity_manhattan.head()

In [ ]:
mandat = 2
print("We look for the most similar tweets to: ")
Similar_years ['Manhattan similarity'] = pd.DataFrame(df_similarity_manhattan[mandat])
Similar_years

In [ ]:
Similar_years.sort_values(by=['Manhattan similarity'] ,ascending=True)

## c - Finding similar documents using cosine similarity
Cosine Similarity is a method of calculating the similarity of two vectors by taking the dot product and dividing it by the magnitudes of each vector, as shown by the illustration below:

In [ ]:
from PIL import Image
import requests
url = "https://miro.medium.com/v2/resize:fit:720/format:webp/1*IhpY-6LYV75983THCpWo-w.png"
Image.open(requests.get(url, stream=True).raw)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

df_similarity_cosine = pd.DataFrame(cosine_similarity(vectors, vectors))

In [ ]:
df_similarity_cosine

In [ ]:
mandat = 2
print("We look for the most yers to: " )
Similar_years ['Cosine similarity'] = pd.DataFrame(df_similarity_cosine[mandat])
Similar_years

In [ ]:
Similar_years.sort_values(by=['Cosine similarity'] ,ascending=False)

In [ ]:
# Create a figure and axes
fig = plt.figure(figsize=(17,10), dpi=70)
ax = fig.add_subplot(111)
# Visualize the matrix with colored squares indicating similarity
ax.matshow(df_similarity_cosine, cmap='rainbow_r', vmin = 0.0, vmax = 1)
# Set regular ticks, one for each book in the collection
ax.set_xticks(range(len(df_results["period"])))
ax.set_yticks(range(len(df_results["period"])))
# Set the tick labels as the book titles
ax.set_xticklabels(df_results["period"])
ax.set_yticklabels(df_results["period"])
# Rotate the labels on the x-axis by 90 degrees
plt.xticks(rotation=90)
plt.show()